## Avila training
 This notebook loads the Avila dataset (https://archive.ics.uci.edu/ml/datasets/Avila), preprocesses it and trains a simple model

In [ ]:
import sys
import os
PROJ_DIR = os.path.realpath(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.join(PROJ_DIR,'src'))



Load data from CSV and save it to a suitable format. This can be skipped if concrete_data.npz is in assets.

In [ ]:
import pandas as pd
import numpy as np

def load_avila(path):
    df = pd.read_csv(path, delimiter=',', header=None)

    labels = df[10]
    x = df.drop(columns=10)

    possible_labels = labels.unique().tolist()
    print(possible_labels)
    print(len(possible_labels))
    y = labels.map(lambda x: possible_labels.index(x))

    return x.to_numpy(), y.to_numpy()
    
x_train, y_train = load_avila(os.path.join(PROJ_DIR, 'assets', 'data', 'avila-tr.txt'))
x_test, y_test = load_avila(os.path.join(PROJ_DIR, 'assets', 'data', 'avila-tr.txt'))

In [ ]:
# Save to assets
np.savez(os.path.join(PROJ_DIR, 'assets', 'data', 'avila'),\
        x_train=x_train,\
        x_test=x_test,\
        y_train=y_train,\
        y_test=y_test)

Load data from file

In [ ]:
file_data = np.load(os.path.join(PROJ_DIR, 'assets', 'data', 'avila.npz'))
x_train = file_data['x_train']
x_test = file_data['x_test']
y_train = file_data['y_train']
y_test = file_data['y_test']

Train a MLP model

In [ ]:
import torch

MODEL_NEURONS = 100
MODEL_EPOCHS= 2000
MODEL_LR = 1.0e-1
MODEL_LABEL_NUM = len(np.unique(y_train))

class MLP(torch.nn.Module):
    def __init__(self, n_neurons):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(x_train.shape[1], n_neurons)
        self.ac1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_neurons, MODEL_LABEL_NUM)
        self.ac2 = torch.nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        logits = self.fc2(x)
        x = self.ac2(logits)
        return x

network = MLP(MODEL_NEURONS)
loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(network.parameters(), lr=MODEL_LR)


x_train_tensor = torch.tensor(x_train).float()
y_train_tensor = torch.tensor(y_train)
x_test_tensor = torch.tensor(x_test).float()
y_test_tensor = torch.tensor(y_test)

for epoch in range(MODEL_EPOCHS):
    optimizer.zero_grad()
    
    preds = network(x_train_tensor)
    label_onehot = torch.zeros(y_train.shape[0], MODEL_LABEL_NUM)
    label_onehot.scatter_(1, y_train_tensor.unsqueeze(1), 1)
    loss_value = loss(preds, label_onehot)
    loss_value.backward()        
    optimizer.step()

    test_preds = network.forward(x_test_tensor)        
    accuracy = (test_preds.argmax(dim=1) == y_test_tensor).float().mean() 
    print(f'Epoch {epoch}/{MODEL_EPOCHS} - Loss: {loss_value.item()} - Test accuracy: {accuracy}')  
    
print(accuracy.item())


In [ ]:
# Save model
torch.save(network.state_dict(), os.path.join(PROJ_DIR,'assets','models','avila-mlp.pth'))